# Árvore B
<em>Material inspirado nas transparências de aula da profa. Vanessa Braganholo e livro do prof Jayme</em>

## Motivação
Arquivos binários grandes
* Busca sequencial é muito custosa
* Se arquivo estiver ordenado pode-se fazer busca binária, mas para arquivos grandes ainda não é eficiente o suficiente

É possível acelerar a busca usando duas técnicas: 
* Acesso via cálculo do endereço do registro (hashing) 
* Acesso via estrutura de dados auxiliar (índice)

## Índice
Índice é uma estrutura de dados que serve para localizar registros no arquivo de dados

Cada entrada do índice contém 
* Valor da chave
* Ponteiro para o arquivo de dados

Pode-se pensar então em dois arquivos: 
* Um de índice
* Um de dados

Isso é eficiente?

### Índice Plano

<img src="./img/indice_plano.png" height="60" width="400">

Se tivermos que percorrer o arquivo de índice sequencialmente para encontrar uma determinada chave, o índice não terá muita utilidade
* Pode-se fazer busca um pouco mais eficiente (ex. busca binária), se o arquivo de índice estiver ordenado
* Mas mesmo assim isso não é o ideal

Para resolver este problema:
* os índices não são estruturas sequenciais, e sim hierárquicas
* os índices não apontam para um registro específico, mas para um bloco de registros (e dentro do bloco é feita busca sequencial) – exige que os registros dentro de um bloco estejam ordenados

<img src="./img/indice_hierarquico.png" height="150" width="500">

## Hierarquia ... árvore!

A maioria das estruturas de índice é implementada por árvores de busca
* Árvores de Busca Binária 
* Árvores AVL
* Árvores de Múltiplos Caminhos

### ÁRVORE DE BUSCA BINÁRIA
Relembrando: Características de uma árvore de busca binária T
* todas as chaves da subárvore da esquerda de T têm valores menores que a chave do nó raiz de T
* todas as chaves da subárvore da direita de T têm valores maiores que a chave do nó raiz de T
* as subárvores esquerda e direita de T também são árvores de busca binária

#### Lembrando:
Altura tende a ser muito grande em relação ao número de nós ou registros que ela contém

Se as chaves a serem incluídas estiverem ordenadas, a árvore degrada-se rapidamente, tornando-se uma lista encadeada:

<img src="./img/abb_desbalanceada.png" alt="Árvore de Busca Binária Desbalanceada" width="200"/>

### ÁRVORES AVL
São árvores binárias balanceadas
* Para qualquer nó da árvore, a altura da subárvore da esquerda não pode diferir em mais de 1 unidade da altura da subárvore da direita

<img src="./img/fb.png" alt="Balanceando Árvores" width="550"/>

##### Árvores AVL são excessivamente altas para uso eficiente como estrutura de índice


### SOLUÇÃO: ÁRVORES DE MÚLTIPLOS CAMINHOS
Características
* Cada nó contém n-1 chaves
* Cada nó contém n filhos
* As chaves dentro do nó estão ordenadas
* As chaves dentro do nó funcionam como separadores para os ponteiros para os filhos do nó

<img src="./img/arv_mult_caminhos.png" width="400" height="200">

#### VANTAGENS
Têm altura bem menor que as árvores binárias 

Ideais para uso como índice de arquivos em disco

Como as árvores são baixas, são necessários poucos acessos em disco até chegar ao ponteiro para o bloco que contém o registro desejado

### Árvores Múltiplos Caminhos: Árvore B e Árvore B+

# Árvore B

Consegue armazenar índice e dados na mesma estrutura (mesmo arquivo físico)

### Características de uma árvore B de ordem d 
* A raiz é uma folha ou tem no mínimo 2 filhos
* Cada nó interno (não folha e não raiz) possui no mínimo d + 1 filhos 
* Cada nó tem no máximo 2d + 1 filhos
* Todas as folhas estão no mesmo nível

Um nó de uma árvore B é também chamado de página

Uma página armazena diversos registros da tabela original 

* Seu tamanho normalmente equivale ao tamanho de uma página em disco


### Outras propriedades
* Seja m o número de chaves de uma página P não folha
* P tem m+1 filhos, P tem entre d e 2d chaves, exceto o nó raiz, que possui entre 1 e 2d chaves
* Em cada página, as chaves estão ordenadas: s1, ..., sm, onde d ≤ m ≤ 2d, exceto para a raiz onde 1 ≤ m ≤ 2d
* P contém m+1 ponteiros p0, p1, ..., pm para os filhos de P
* Nas páginas correspondentes às folhas, esses ponteiros apontam para NULL
* Os nós também armazenam, além da chave sk, os dados (Ik) relativos àquela chave


Seja uma página P com m chaves:
* para qualquer chave y pertencente à primeira página apontada por P (ou seja, apontada por p0), y < s1
* para qualquer chave y pertencente à página apontada por pk, 1 ≤ k ≤ m-1, sk < y < sk+1 
* para qualquer chave y pertencente à página apontada por pm, y > sm

<img src="./img/estrutura_pag.png" width="400">

#### Implementação em C

```
typedef struct No {
    int m; //quantidade de chaves armazenadas no nó struct No *pont_pai; //pt para o nó pai
    int *s; //array de chaves
    struct No **p; //pt para array de pt p/ os filhos
} TNo;
```

1. Implemente o nó de uma árvore B em python:

In [ ]:
class BTreeNode:
    def __init__(self, leaf=False):
        self.leaf = leaf  # Nó Folha
        self.keys = []  # Valores
        self.child = []  # Nós Filhos

## BUSCA DE UMA CHAVE X EM ÁRVORE B QUE INDEXA ARQUIVO EM DISCO
 1. Inicie lendo a raiz da árvore a partir do disco
 2. Procure x dentro do nó lido (pode ser usada busca binária, pois as chaves estão ordenadas dentro do nó)
     
     a) Se encontrou, encerra a busca;
     
     b) Caso contrário, continue a busca, lendo o filho correspondente, a partir do disco
     
     
 3. Continue a busca até que x tenha sido encontrado ou que a busca tenha sido feita em uma folha da árvore (retorna o último nó pesquisado – nó onde a chave está ou deveria estar)
 
#### Buscando as chaves 240, 76 e 85 na árvore

<img src="./img/exemplo_busca_arvb.png" width="900">

#### Implementação em C da Função Busca

```
TNo *busca(TNo *no, int ch) { 
    if (no != NULL) {
        int i = 0;
        while (i < no->m && ch > no->s[i]) {
            i++; 
        }
        if (i < no->m && ch == no->s[i]) { 
            return no; // encontrou chave
        } else if (no->p[i] != NULL) { 
            return busca(no->p[i], ch);
        } else 
            return no; //nó era folha -- não existem mais nós a buscar, então retorna o nó onde a chave deveria estar
    } else 
        return NULL; //nó é NULL, não há como buscar }
```

2. Implemente a função busca em python

In [ ]:
    def search(self, key, node=None):

        if node is not None:
            i = 0
            while i < len(node.keys) and key > node.keys[i][0]:
                i += 1
            if i < len(node.keys) and key == node.keys[i][0]:
                return node, i
            elif node.leaf:
                return None
            else:
                return self.search(key, node.child[i])
        else:
            return self.search(key, self.root)

## INSERÇÃO
Para inserir um registro de chave x na árvore B
* Executar o algoritmo de busca
* Se chave está no nó retornado pela busca (é preciso checar) 
 * Inserção é inválida
* Se chave não está no nó retornado pela busca: 
 * Inserir a chave no nó retornado pela busca
 
 
#### Inserindo a chave 32
<img src="./img/arvb_insercao.png" width="800">

<img src="./img/arvb_insercao1.png" width="800">


##### Inserção sempre ocorre nas folhas

#### Inserindo a chave 85 (página excede tamanho máximo de 2d + 1 chaves)
<img src="./img/arvb_insercao2.png" width="800">

#### Página Cheia!!!

É necessário reorganizar as páginas

Ao inserir uma chave em uma página cheia, sua estrutura ficaria da seguinte forma (omitindo Ik para simplificar)

* p0, (s1, p1), (s2,p2), ..., (sd,pd), (sd+1,pd+1), ..., (s2d+1, p2d+1) 

<img src="./img/pagina_arvb.png" width="400">

### SOLUÇÃO
Particionar a página em 2

* Na página P permanecem d entradas
* Entrada d+1 sobe para o pai
* Alocar outra página, Q, e nela alocar as outras d entradas

Após o particionamento 
* Estrutura da página P:
 * p0, (s1, p1), (s2, p2), ..., (sd, pd) 
* Estrutura da página Q:
 * pd+1, (sd+2, pd+2) ..., (s2d+1, p2d+1)
 
#### ALOCAÇÃO DE SD+1
O nó W, agora também pai de Q, receberá a nova entrada (sd+1, pt) 
* pt aponta para a nova página Q

Se não houver mais espaço livre em W, o processo de particionamento também é aplicado a W

#### PARTICIONAMENTO

Observação importante: particionamento se propaga para os pais dos nós, podendo, eventualmente, atingir a raiz da árvore

O particionamento da raiz é a única forma de aumentar a altura da árvore

### PROCEDIMENTO DE INSERÇÃO

1. Aplicar o procedimento busca, verificando a validade da inserção
2. Se a inserção é válida, realizar inserção no nó F retornado pela busca
3. Verificar se nó F precisa de particionamento. Se sim, propagar o particionamento enquanto for necessário.

### Exemplo de inserção com particionamento: Chave 85

<img src="./img/arvb_insercao2.png" width="800">

<img src="./img/arvb_insercao3.png" width="800">

<img src="./img/arvb_insercao4.png" width="800">

<img src="./img/arvb_insercao5.png" width="800">


### Exemplo de inserção com propagação: Chave 73

<img src="./img/arvb_insercao6.png" width="800">

<img src="./img/arvb_insercao7.png" width="800">

<img src="./img/arvb_insercao8.png" width="800">

<img src="./img/arvb_insercao9.png" width="800">

<img src="./img/arvb_insercao10.png" width="800">

<img src="./img/arvb_insercao11.png" width="800">

<img src="./img/arvb_insercao12.png" width="800">


#### Como seria inserir as chaves 57, 71, 72, 73???



### PROCEDIMENTO DE EXCLUSÃO

Duas situações possíveis:
* A entrada x está em um nó folha
 * Neste caso, simplesmente remover a entrada x

* A entrada x não está em um nó folha
 * Substituir x pela chave y imediatamente maior
 * Note que y necessariamente pertence a uma folha, pela forma como a árvore B é estruturada
 
<img src="./img/arvb_exclusao1.png" width="800">

<img src="./img/arvb_exclusao2.png" width="800">

<img src="./img/arvb_exclusao3.png" width="800">

#### Caso 2 - Exclusão da chave 9

<img src="./img/arvb_exclusao4.png" width="800">

<img src="./img/arvb_exclusao5.png" width="800">

<img src="./img/arvb_exclusao6.png" width="800">

#### Caso 3 - E se o nó ficar com menos de d chaves (não é permitido)? -- Concatenação ou Redistribuição

#### Exemplo 
<img src="./img/arvb_exclusao7.png" width="800">

## Concatenação

Duas páginas P e Q são irmãs adjacentes se têm o mesmo pai W e são apontadas por dois ponteiros adjacentes em W

P e Q podem ser concatenadas se: 
* são irmãs adjacentes; e
* juntas possuem menos de 2d chaves

### OPERAÇÃO DE CONCATENAÇÃO DE P E Q
Agrupar as entradas de Q em P

Em W, pegar a chave si que está entre os ponteiros que apontam para P e Q, e transferi-la para P

Em W, eliminar o ponteiro pi (ponteiro que ficava junto à chave si que foi transferida)

#### Exemplo

<img src="./img/arvb_exclusao7.png" width="800">

<img src="./img/arvb_exclusao8.png" width="800">

<img src="./img/arvb_exclusao9.png" width="800">

<img src="./img/arvb_exclusao10.png" width="800">

<img src="./img/arvb_exclusao11.png" width="800">

<img src="./img/arvb_exclusao12.png" width="800">

<img src="./img/arvb_exclusao13.png" width="800">

<img src="./img/arvb_exclusao14.png" width="800">

<img src="./img/arvb_exclusao15.png" width="800">

<img src="./img/arvb_exclusao16.png" width="800">

<img src="./img/arvb_exclusao17.png" width="800">

## Redistribuição

Ocorre quando a soma das entradas de P e de seu irmão adjacente Q é maior ou igual a 2d

Concatenar P e Q

* Isso resulta em um nó P com mais de 2d chaves, o que não é permitido 
* Particionar o nó concatenado, usando Q como novo nó
* Essa operação não é propagável: o nó W, pai de P e Q, é alterado, mas seu número de chaves não é modificado

### Exemplo:

<img src="./img/arvb_exclusao18.png" width="800">

<img src="./img/arvb_exclusao19.png" width="800">

<img src="./img/arvb_exclusao20.png" width="800">

## E QUANDO AS DUAS ALTERNATIVAS SÃO POSSÍVEIS?

Quando for possível usar concatenação ou redistribuição (porque o nó possui 2 nós adjacentes, cada um levando a uma solução diferente), optar pela <b>redistribuição</b>

* Ela é menos custosa, pois não se propaga
* Ela evita que o nó fique cheio, deixando espaço para futuras inserções


# Exercício

3. Desenhar uma árvore B de ordem 3 que contenha as seguintes chaves: 1, 3, 6, 8, 14, 32, 36, 38, 39, 41, 43

Dica: começar com uma árvore B vazia e ir inserindo uma chave após a outra 

Relembrando características de uma árvore B de ordem d

* A raiz é uma folha ou tem no mínimo 2 filhos
* Cada nó interno (não folha e não raiz) possui no mínimo d + 1 filhos 
* Cada nó tem no máximo 2d + 1 filhos
* Todas as folhas estão no mesmo nível

4. Sobre a árvore resultante do exercício anterior, realizar as seguintes operações:
(a) Inserir as chaves 4, 5, 42, 2, 7
(b) Sobre o resultado do passo (a), excluir as chaves 14, 32